In [2]:
from dfs.DFReader import get_asserted_spells_df

spells_df = get_asserted_spells_df(verbose=True)

Spells: 100%|██████████| 202/202 [00:00<00:00, 268.39it/s]


In [6]:
from dfs.DFFilter import filter_df
from spell.SpellPrinter import print_spell, print_spells_by_name, print_spells_for_df

my_filter = {
    "classes": ['guerreiro'],
    'escola': ['musical', 'elemental'],
    'elementos': ['fogo'],
    "nivel": [0, 1],
}
filtered_df = filter_df(spells_df, my_filter)

print_spells_for_df(filtered_df)

<span style='color:None;font-size:None'>**Ataque Elemental _(Custom)_** - truque<br></span><span style='color:None;font-size:13px'>&emsp;**Escola(s):** elemental, musical, ilusionista, necromancia, psíquica (_água, terra, fogo, ar, luz, sombras, relâmpago, metal, veneno_)<br>&emsp;**Tempo conjuração:** 1 ação<br>&emsp;**Alcance:** 18 metros<br>&emsp;**Componentes:** S<br>&emsp;**Mana:** 0<br>&emsp;**Duração:** instantânea<br>&emsp;**Dano**: 1d6<br>&emsp;**Dmg/effect:** elemental<br></span><span style='color:None;font-size:None'>Essa é a magia mais básica de dano que qualquer mago sabe utilizar e utiliza uma quantidade de mana praticamente nula (se sua mana estiver abaixo do limite, você ainda deve fazer testes de exaustão).<br>&emsp;Você utiliza sua escola e elemento para causa 1d6 de dano no inimigo.<br>&emsp;O dano dessa magia aumenta em 1d6 quando você alcança o 5° nível (2d6), 11° nível (3d6) e 17° nível (4d6).<br></span><span style='color:gray;font-size:11px'>[dano]<br></span><span style='color:gray;font-size:11px'>[arqueiro, bardo, guerreiro, ladino, mago, monge, xamã]<br></span><span style='color:gray;font-size:10px'>_Custom_</span>

<span style='color:None;font-size:None'>**Chicote _(Lightning Lure)_** - truque<br></span><span style='color:None;font-size:13px'>&emsp;**Escola(s):** elemental (_relâmpago, luz, sombras, metal, terra, fogo_)<br>&emsp;**Tempo conjuração:** 1 ação<br>&emsp;**Alcance:** 4,5 metros<br>&emsp;**Componentes:** VS<br>&emsp;**Mana:** 150<br>&emsp;**Duração:** instantânea<br>&emsp;**Dano**: 1d8<br>&emsp;**Attack/Save:** STR Save<br>&emsp;**Dmg/effect:** elemental<br></span><span style='color:None;font-size:None'>Você cria um chicote elemental que golpeia uma criatura a sua escolha a sua vista a até 4,5m de você. O alvo deve ser bem-sucedido em uma salvaguarda de Força ou é puxado por até 3m em linha reta na sua direção, e então recebe 1d8 de dano elemental se estiver a até 1,5m de você.<br>&emsp;O dano dessa magia aumenta em 1d8 quando você alcança o 5º nível (2d8), 11º nível (3d8), e 17º nível (4d8).<br></span><span style='color:gray;font-size:11px'>[controle, dano]<br></span><span style='color:gray;font-size:11px'>[guerreiro, mago, monge]<br></span><span style='color:gray;font-size:10px'>_Tasha_</span>

<span style='color:None;font-size:None'>**Controlar Chamas _(Control Flames)_** - truque<br></span><span style='color:None;font-size:13px'>&emsp;**Escola(s):** elemental (_fogo, sombras_)<br>&emsp;**Tempo conjuração:** 1 ação<br>&emsp;**Alcance:** 18 metros<br>&emsp;**Componentes:** S<br>&emsp;**Mana:** 100 (_+ 5 a cada 5 minutos_)<br>&emsp;**Duração:** instantânea ou 1 hora<br></span><span style='color:None;font-size:None'>Você escolhe uma chama não mágica que você possa ver, dentro do alcance, e que ocupe até um cubo de 1,5 metros. Você afeta ela de uma das seguintes formas:<br>&emsp;• Você instantaneamente expande a chama em 1,5 metro em uma direção, considerando que exista madeira ou outro combustível no local novo.<br>&emsp;• Você instantaneamente extingue as chamas dentro do cubo.<br>&emsp;• Você dobra ou reduz à metade a área de luz plena e de penumbra emitida pela chama, muda a cor dela, ou ambos. As mudanças duram por 1 hora.<br>&emsp;• Você faz com que formas simples - como um forma imprecisa de uma criatura, objeto inanimado ou local - apareçam dentro das chamas e se animem como você quiser. As formas duram por 1 hora.<br>&emsp;Se você conjurar essa magia diversas vezes , você pode ter até três dos seus efeitos não instantâneos ativos ao mesmo tempo, e você pode dissipar um efeito desses com uma ação.<br></span><span style='color:gray;font-size:11px'>[controle, utilidade]<br></span><span style='color:gray;font-size:11px'>[arqueiro, bardo, guerreiro, ladino, mago, monge, xamã]<br></span><span style='color:gray;font-size:10px'>_Xanathar_</span>

<span style='color:None;font-size:None'>**Criar Chamas _(Produce Flame)_** - truque<br></span><span style='color:None;font-size:13px'>&emsp;**Escola(s):** elemental (_fogo, luz, sombras_)<br>&emsp;**Tempo conjuração:** 1 ação<br>&emsp;**Alcance:** pessoal<br>&emsp;**Componentes:** VS<br>&emsp;**Mana:** 50 (_+ 5 por minuto_)<br>&emsp;**Duração:** 10 minutos<br>&emsp;**Dano**: 1d8<br>&emsp;**Attack/Save:** distância<br>&emsp;**Dmg/effect:** fogo<br></span><span style='color:None;font-size:None'>Uma chama tremulante aparece na sua mão. A chama permanece ai pela duração e não machuca nem você nem seu equipamento. A chama emite luz plena num raio de 3 metros e penumbra por 3 metros adicionais. A magia acaba se você dissipa-la usando uma ação ou se conjura-la novamente.<br>&emsp;Você pode, também, atacar com a chama, no entanto, fazer isso acaba com a magia. Quando você conjura essa magia ou com uma ação em um turno posterior, você pode arremessar a chama numa criatura a até 9 metros de você. Faça um ataque à distância com magia. Se atingir, o alvo sofre 1d8 de dano elemental.<br>&emsp;[Sombras] Todos os alcances de iluminação descritos anteriormente são metade para um Mago das Sombras.<br>&emsp;Você gasta 10 de mana por minuto até a magia acabar.<br>&emsp;O dano dessa magia aumenta em 1d8 quando você alcança o 5° nível (2d8), 11° nível (3d8) e 17° nível (4d8).<br></span><span style='color:gray;font-size:11px'>[dano, distância, utilidade]<br></span><span style='color:gray;font-size:11px'>[arqueiro, bardo, guerreiro, ladino, mago, monge, xamã]<br></span><span style='color:gray;font-size:10px'>_LDJ_</span>

<span style='color:None;font-size:None'>**Criar Fogueira Instantânea _(Create Bonfire)_** - truque<br></span><span style='color:None;font-size:13px'>&emsp;**Escola(s):** elemental (_fogo, sombras_)<br>&emsp;**Tempo conjuração:** 1 ação<br>&emsp;**Alcance:** 18 metros<br>&emsp;**Componentes:** VS<br>&emsp;**Mana:** 150<br>&emsp;**Duração:** instantânea<br>&emsp;**Dano**: 1d8<br>&emsp;**Attack/Save:** DEX Save<br>&emsp;**Dmg/effect:** fogo<br></span><span style='color:None;font-size:None'>Você cria uma fogueira no solo em um ponto que você possa ver, dentro do alcance. A fogueira preenche um cubo de 1,5 metros. Qualquer criatura no espaço da fogueira quando você a conjura deve ser bem sucedida num teste de resistência de Destreza ou sofrerá 1d8 de dano de fogo.<br>&emsp;O dano da magia aumenta em 1d8 quando você alcança o 5º nível (2d8), 11º nível (3d8) e 17º nível (4d8).<br></span><span style='color:gray;font-size:11px'>[dano]<br></span><span style='color:gray;font-size:11px'>[guerreiro, ladino, monge]<br></span><span style='color:gray;font-size:10px'>_Xanathar_</span>

...

<span style='color:None;font-size:None'>**Rompante de Espadas _(Sword Burst)_** - truque<br></span><span style='color:None;font-size:13px'>&emsp;**Escola(s):** elemental (_água, terra, ar, fogo, sombras, luz, metal_)<br>&emsp;**Tempo conjuração:** 1 ação<br>&emsp;**Alcance:** pessoal<br>&emsp;**Componentes:** VSM _(um pouco de pó de prata)_<br>&emsp;**Mana:** 100<br>&emsp;**Duração:** instantânea<br>&emsp;**Dano**: 1d6<br>&emsp;**Attack/Save:** DEX Save<br>&emsp;**Dmg/effect:** fogo<br></span><span style='color:None;font-size:None'>Você cria um círculo momentâneo de lâminas elementais que giram ao seu redor. Todas as outras criaturas a até 1,5m de você devem ser bem-sucedidas em uma salvaguarda de Destreza ou recebem 1d6 de dano elemental.<br>&emsp;O dano da magia aumenta em 1d6 quando você alcança o 5º nível (2d6), 11 nível (3d6), e 17º nível (4d6).<br></span><span style='color:gray;font-size:11px'>[dano]<br></span><span style='color:gray;font-size:11px'>[arqueiro, bardo, guerreiro, ladino, mago, monge, xamã]<br></span><span style='color:gray;font-size:10px'>_Tasha_</span>

<span style='color:None;font-size:None'>**Destruição Lancinante _(Wrathful Smite)_** lvl 1<br></span><span style='color:None;font-size:13px'>&emsp;**Escola(s):** elemental (_fogo, luz, sombras_)<br>&emsp;**Tempo conjuração:** 1 ação bônus<br>&emsp;**Alcance:** pessoal<br>&emsp;**Componentes:** V<br>&emsp;**Mana:** 400 (_+ 10 por turno_)<br>&emsp;**Duração:** concentração, até 1 minuto<br>&emsp;**Dano**: 1d6<br>&emsp;**Attack/Save:** corpo-a-corpo<br>&emsp;**Dmg/effect:** elemental<br></span><span style='color:None;font-size:None'>Da próxima vez que você atingir uma criatura com um ataque corpo-a-corpo com arma enquanto essa magia durar, sua arma flameja com intensas chamas e o ataque causa 1d6 de dano elemental extra ao alvo, fazendo-o incendiar pelas chamas. No início de cada turno dele, até a magia acabar, o alvo deve realizar um teste de resistência de Constituição. Se falhar na resistência, ele sofre 1d6 de dano de fogo. Se passar na resistência, a magia acaba. Se o alvo ou uma criatura a 1,5 metro dele usar uma ação para apagar as chamas ou se algum outro efeito extinguir as chamas (como submergir o alvo em água), a magia acaba.<br>&emsp;**Melhorar Magia**. Você pode gastar 500 de mana adicionais e 10 por turno para adicionar 1d6 no dano até um máximo de 5d6 adicionais.<br></span><span style='color:gray;font-size:11px'>[arma, corpo-a-corpo, dano]<br></span><span style='color:gray;font-size:11px'>[arqueiro, bardo, guerreiro, ladino, mago, monge, xamã]<br></span><span style='color:gray;font-size:10px'>_LDJ_</span>

<span style='color:None;font-size:None'>**Encantar Arma (Auxílio Divino) _(Searing Smite)_** lvl 1<br></span><span style='color:None;font-size:13px'>&emsp;**Escola(s):** elemental (_água, terra, fogo, ar, luz, sombras, relâmpago, metal, veneno_)<br>&emsp;**Tempo conjuração:** 1 ação bônus<br>&emsp;**Alcance:** pessoal<br>&emsp;**Componentes:** VS<br>&emsp;**Mana:** 300 (_+ 20 por turno_)<br>&emsp;**Duração:** concentração, até 1 minuto<br>&emsp;**Dano**: 1d4<br>&emsp;**Dmg/effect:** elemental<br></span><span style='color:None;font-size:None'>Você envolve sua arma com seu elemento. Até o final da magia, seus ataques com aruma causa 1d4 de dano elemento extra ao atingirem.<br>&emsp;**Melhorar Magia**. Você pode gastar mais 500 de mana e mais 30 de mana por turno para transformar o d4 em d6. Você pode gastar mais 1500 de mana e mais 50 de mana por turno para transformar o d6 em d8. Você pode gastar mais 5000 de mana e mais 100 de mana por turno para transformar o d8 em d10.<br></span><span style='color:gray;font-size:11px'>[arma, buff, dano]<br></span><span style='color:gray;font-size:11px'>[arqueiro, bardo, guerreiro, ladino, mago, monge, xamã]<br></span><span style='color:gray;font-size:10px'>_LDJ_</span>

<span style='color:None;font-size:None'>**Flecha Luminosa _(Custom)_** lvl 1<br></span><span style='color:None;font-size:13px'>&emsp;**Escola(s):** elemental (_fogo, luz, relâmpago_)<br>&emsp;**Tempo conjuração:** 1 ação<br>&emsp;**Alcance:** pessoal<br>&emsp;**Componentes:** VSM _(uma pata de vagalume)_<br>&emsp;**Mana:** 350<br>&emsp;**Duração:** instantânea<br>&emsp;**Dano**: 1d4<br>&emsp;**Dmg/effect:** perfurante<br></span><span style='color:None;font-size:None'>Você encanta uma munição de arma à distância e faz uma jogada de ataque contra um alvo dentro do alcance normal da arma. Independente de acertar ou errar, você cria uma explosão luminosa que cria uma esfera de 3m de luz plena e 3m de penumbra com foco no alvo. Em um acerto, você dá 1d4 de dano perfurante extra.<br></span><span style='color:gray;font-size:11px'>[arma, dano, detecção, distância]<br></span><span style='color:gray;font-size:11px'>[arqueiro, guerreiro]<br></span><span style='color:gray;font-size:10px'>_Custom_</span>

<span style='color:None;font-size:None'>**Raio de Caos _(Chaos Boat)_** lvl 1<br></span><span style='color:None;font-size:13px'>&emsp;**Escola(s):** elemental (_água, terra, fogo, ar, luz, sombras, relâmpago, metal, veneno_)<br>&emsp;**Tempo conjuração:** 1 ação<br>&emsp;**Alcance:** 36 metros<br>&emsp;**Componentes:** VS<br>&emsp;**Mana:** 600<br>&emsp;**Duração:** instantânea<br>&emsp;**Dano**: 2d8 + 1d6<br>&emsp;**Attack/Save:** distância<br>&emsp;**Dmg/effect:** elemental<br></span><span style='color:None;font-size:None'>Você conjura uma massa ondulante de energia caótica elemental em uma criatura no alcance. Faça um ataque à distância com magia contra o alvo. Se atingir, o alvo leva 2d8 + 1d6 de dano elemental.<br>&emsp;Se os dois d8 rolados possuírem o mesmo valor, então a energia caótica salta para um alvo dentro de 9 metros de distância a sua escolha. Faça uma nova rolagem de ataque à distância contra o novo e role novamente o dano, fazendo com que a magia possa novamente saltar para outro alvo.<br>&emsp;Alternativamente, você pode utilizar uma arma corpo-a-corpo para fazer o ataque. Sua arma se enche dessa energia caótica e o efeito é o mesmo, mas a diferença é que o alvo inicial sofre 2d8 + o dano que sua arma possui naturalmente.<br>&emsp;**Melhorar Magia.** Para cada 500 adicionais de mana gasto, o dano da magia aumenta em 1d6, com um limite de 5d6 extras.<br></span><span style='color:gray;font-size:11px'>[arma, corpo-a-corpo, dano]<br></span><span style='color:gray;font-size:11px'>[arqueiro, bardo, guerreiro, ladino, mago, xamã]<br></span><span style='color:gray;font-size:10px'>_Xanathar_</span>

In [3]:
filtered_df

,nome,name,nivel,escola,ritual,elementos,tempo_conjuracao,alcance_area,componentes,mana,duracao,attack_save,dmg_effect,dmg,classes,tags,descricao,source,mana_adicional,magia_rara
18,Infestação,Infestation,0,"[invocação, necromancia]",False,[],1 ação,9 metros,VSM (uma pulga viva),100,instantânea,CON Save,veneno,1d6,"[arqueiro, mago, bardo, ladino, xamã]","[dano, controle]","Você provoca uma nuvem de ácaros, pulgas e out...",Xanathar,N/A,False
28,Mãos Mágicas,Mage Hand,0,"[necromancia, invocação]",False,[],1 ação,9 metros,VS,150,instantânea,N/A,N/A,N/A,"[arqueiro, ladino, guerreiro, monge, bardo, ma...",[utilidade],"Uma mão espectral flutuante aparece num ponto,...",LDJ,N/A,False
62,Convocar Familiar,Find Familiar,1,[invocação],False,[],1 hora,3 metros,"VSM (carvão, incenso e ervas no valor de 10 po...",700,instantânea,N/A,N/A,N/A,"[arqueiro, mago, xamã]","[dano, utilidade, exploração]","Você adquire os serviços de uma familiar, um e...",LDJ,N/A,False
114,Servo Invisível,Unseen Servant,1,[invocação],True,[],1 ação,18 metros,VSM (um pedaço de barbante e um pouco de madeira),200,1 hora,N/A,N/A,N/A,"[arqueiro, ladino, guerreiro, bardo, monge, ma...","[exploração, utilidade]","Essa magia cria uma força invisível, sem mente...",LDJ,N/A,False
138,Convocar Montaria,Find Steed,2,"[invocação, necromancia, espiritual]",False,[],10 minutos,9 metros,VS,1400,instantânea,N/A,N/A,N/A,"[arqueiro, guerreiro, bardo, mago]","[dano, comunicação, exploração]",Você convoca um espírito que assume a forma de...,LDJ,N/A,True
151,Espírito Curativo,Healing Spirits,2,"[elemental, invocação, espiritual]",False,"[água, terra, luz]",1 ação bônus,18 metros,VS,700,"concentração, até 1 minuto",N/A,N/A,1d6,"[bardo, monge, xamã]",[cura],Você invoca um espírito de natureza para confo...,LDJ,15 por turno,False
160,Invocar Fera,Summon Beast,2,"[elemental, invocação]",False,"[água, terra, fogo, ar]",1 ação,18 metros,V,800,1 hora,N/A,N/A,N/A,"[bardo, mago, feiticeiro, bruxo, xamã]",[dano],Você convoca um espírito feral. Ele se manifes...,Tasha,N/A,False
168,Mensageiro Animal,Animal Messenger,2,"[elemental, invocação, musical]",True,[terra],1 ação,9 metros,VSM (um punhado de comida),650,24 horas,N/A,N/A,N/A,"[arqueiro, guerreiro, bardo, monge, mago, xamã]","[utilidade, comunicação]","Através dessa magia, você usa um animal para e...",LDJ,N/A,False
184,Rebanho de Familiares,Flock of Familiars,2,[invocação],False,[],1 minuto,toque,VS,1400,"concentração, até 1 hora",N/A,N/A,N/A,"[arqueiro, ladino, guerreiro, bardo, monge, ma...","[comunicação, detecção, exploração]",Você temporariamente invoca três familiares - ...,Etc.,50 a cada 5 minutos,False
188,Sentido Bestial,Beast Sense,2,"[elemental, musical, invocação]",False,[terra],1 ação,toque,S,500,"concentração, até 1 hora",N/A,N/A,N/A,"[arqueiro, ladino, guerreiro, bardo, monge, ma...","[exploração, toque]",Você toca uma besta voluntária. Pela duração d...,LDJ,20 a cada 5 minutos,False


In [4]:
from IPython.display import display


def get_mana_stats(spells_df):
    display(spells_df.mana.value_counts().sort_index(ascending=False))
    print(f"Mean =\t{filtered_df.mana.mean():.3f}")
    print(f"Std  =\t{filtered_df.mana.std():.3f}")


get_mana_stats(filtered_df)


1400    2
800     1
700     2
650     1
500     1
200     1
150     1
100     1
Name: mana, dtype: int64

Mean =	660.000
Std  =	462.961


In [5]:
import dice.DiceCalculator as dice

dice.get_dice_formula_mean('4d8+2')

20.0